In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np


2024-05-30 16:00:21.811819: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 16:00:21.812002: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 16:00:21.969397: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Define dataset directories
dataset_dir = 'ishafolk/folk-art-dataset-main'


# Create directories for train, validation, and test splits
base_dir = 'folk-art-data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get class names

class_names = [c for c in sorted(os.listdir(dataset_dir)) if c != "README.md"]

for c in class_names:
    os.makedirs(os.path.join(train_dir, c), exist_ok=True)
    os.makedirs(os.path.join(val_dir, c), exist_ok=True)
    os.makedirs(os.path.join(test_dir, c), exist_ok=True)

In [5]:

def split_data(source_dir, train_dir, val_dir, test_dir, split_ratio=(0.7, 0.15, 0.15)):
    for category in os.listdir(source_dir):
        category_path = os.path.join(source_dir, category)
        if not os.path.isdir(category_path):  # Check if it's a directory
            continue
        images = os.listdir(category_path)
        train, temp = train_test_split(images, test_size=(1 - split_ratio[0]), random_state=42)
        val, test = train_test_split(temp, test_size=0.5, random_state=42)

        for img in train:
            shutil.copy(os.path.join(category_path, img), os.path.join(train_dir, category))
        for img in val:
            shutil.copy(os.path.join(category_path, img), os.path.join(val_dir, category))
        for img in test:
            shutil.copy(os.path.join(category_path, img), os.path.join(test_dir, category))

# Split the dataset
split_data(dataset_dir, train_dir, val_dir, test_dir)

In [6]:
print("Class names:", class_names)

Class names: ['Aipan Art (Uttarakhand)', 'Assamese Miniature Painting (Assam)', 'Basholi Painting (Jammu and Kashmir)', 'Bhil Painting (Madhya Pradesh)', 'Chamba Rumal (Himachal Pradesh)', 'Cheriyal Scroll Painting (Telangana)', 'Dokra Art(West Bengal)', 'Gond Painting (Madhya Pradesh)', 'Kalamkari Painting (Andra Pradesh and Telangana)', 'Kalighat Painting (West Bengal)', 'Kangra Painting (Himachal Pradesh)', 'Kerala Mural Painting (Kerala)', 'Kondapalli Bommallu (Andra Pradesh)', 'Kutch Lippan Art (Gujarat)', 'Leather Puppet Art (Andra Pradesh)', 'Madhubani Painting (Bihar)', 'Mandala Art', 'Mandana Art (Rajasthan)', 'Mata Ni Pachedi (Gujarat)', 'Meenakari Painting (Rajasthan)', 'Mughal Paintings', 'Mysore Ganjifa Art (Karnataka)', 'Pattachitra Painting (Odisha and Bengal)', 'Patua Painting (West Bengal)', 'Pichwai Painting (Rajasthan)', 'Rajasthani Miniature Painting (Rajasthan)', 'Rogan Art from Kutch (Gujarat)', 'Sohrai Art (Jharkhand)', 'Tikuli Art (Bihar)', 'Warli Folk Painting 

In [23]:
# Data augmentation and generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    classes=class_names  # Use the actual class names, excluding "readme"
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    classes=class_names  # Use the actual class names, excluding "readme"
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    classes=class_names  # Use the actual class names, excluding "readme"
)


# Build and compile EfficientNetB0 model
base_model_efficientnet = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model_efficientnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(30, activation='softmax')(x)  # Output layer with 30 units

model_efficientnet = Model(inputs=base_model_efficientnet.input, outputs=predictions)
model_efficientnet.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint_efficientnet = ModelCheckpoint('efficientnet_best_model.weights.h5', monitor='val_accuracy', save_best_only=True, mode='max', save_weights_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train EfficientNet model
history_efficientnet = model_efficientnet.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[checkpoint_efficientnet, early_stopping]
)

Found 22601 images belonging to 30 classes.
Found 4845 images belonging to 30 classes.
Found 4864 images belonging to 30 classes.
Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  1/707 ━━━━━━━━━━━━━━━━━━━━ 18:23:47 94s/step - accuracy: 0.0312 - loss: 3.5219

I0000 00:00:1717073770.004489     169 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


157/707 ━━━━━━━━━━━━━━━━━━━━ 3:17 359ms/step - accuracy: 0.1968 - loss: 3.0453

/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


707/707 ━━━━━━━━━━━━━━━━━━━━ 419s 461ms/step - accuracy: 0.4363 - loss: 2.1312 - val_accuracy: 0.5534 - val_loss: 1.6079
Epoch 2/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 327s 378ms/step - accuracy: 0.7564 - loss: 0.8341 - val_accuracy: 0.7670 - val_loss: 0.8210
Epoch 3/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 267s 374ms/step - accuracy: 0.8166 - loss: 0.6182 - val_accuracy: 0.7959 - val_loss: 0.7238
Epoch 4/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 266s 372ms/step - accuracy: 0.8632 - loss: 0.4613 - val_accuracy: 0.8091 - val_loss: 0.6962
Epoch 5/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 274s 383ms/step - accuracy: 0.8882 - loss: 0.3714 - val_accuracy: 0.8194 - val_loss: 0.6475
Epoch 6/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 269s 376ms/step - accuracy: 0.9054 - loss: 0.3084 - val_accuracy: 0.8332 - val_loss: 0.6597
Epoch 7/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 266s 373ms/step - accuracy: 0.9219 - loss: 0.2519 - val_accuracy: 0.8388 - val_loss: 0.6357
Epoch 8/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 267s 374ms/step - accuracy: 0.9330 - loss: 0.21

In [28]:
# Build and compile ResNet50 model
base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model_resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(30, activation='softmax')(x)

model_resnet = Model(inputs=base_model_resnet.input, outputs=predictions)
model_resnet.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for ResNet
# Callbacks for ResNet
checkpoint_resnet = ModelCheckpoint('resnet_best_model.weights.h5', monitor='val_accuracy', save_best_only=True, mode='max', save_weights_only=True)
# Train ResNet model
history_resnet = model_resnet.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[checkpoint_resnet, early_stopping]
)


Epoch 1/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 367s 421ms/step - accuracy: 0.5133 - loss: 1.7566 - val_accuracy: 0.2698 - val_loss: 2.9835
Epoch 2/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 266s 373ms/step - accuracy: 0.7742 - loss: 0.7681 - val_accuracy: 0.7459 - val_loss: 0.9118
Epoch 3/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 265s 371ms/step - accuracy: 0.8300 - loss: 0.5643 - val_accuracy: 0.7269 - val_loss: 1.0667
Epoch 4/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 267s 374ms/step - accuracy: 0.8669 - loss: 0.4430 - val_accuracy: 0.7595 - val_loss: 0.9867
Epoch 5/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 269s 376ms/step - accuracy: 0.8867 - loss: 0.3649 - val_accuracy: 0.7860 - val_loss: 0.8653
Epoch 6/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 266s 372ms/step - accuracy: 0.9057 - loss: 0.3029 - val_accuracy: 0.7853 - val_loss: 0.9133
Epoch 7/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 268s 376ms/step - accuracy: 0.9117 - loss: 0.2725 - val_accuracy: 0.8058 - val_loss: 0.8373
Epoch 8/50
707/707 ━━━━━━━━━━━━━━━━━━━━ 267s 375ms/step - accuracy: 0.9225 -

In [29]:
# Load best models and evaluate
model_efficientnet.load_weights('efficientnet_best_model.weights.h5')
model_resnet.load_weights('resnet_best_model.weights.h5')

efficientnet_eval = model_efficientnet.evaluate(test_generator)
print(f'EfficientNet Test Accuracy: {efficientnet_eval[1]*100:.2f}%')

resnet_eval = model_resnet.evaluate(test_generator)
print(f'ResNet Test Accuracy: {resnet_eval[1]*100:.2f}%')


152/152 ━━━━━━━━━━━━━━━━━━━━ 10s 64ms/step - accuracy: 0.8705 - loss: 0.6269
EfficientNet Test Accuracy: 86.31%
152/152 ━━━━━━━━━━━━━━━━━━━━ 9s 62ms/step - accuracy: 0.8315 - loss: 0.7460
ResNet Test Accuracy: 82.73%


In [31]:
model_efficientnet.save('efficientnet_model.h5')
